Appendix
* Target sentences: `results/FinancialPhraseBank_DSAll_k-folds/predictions_senti-dd.csv`
* Senti-DD Lexicon: `data/DS50_Entire/Senti-DD.csv`
* Loughran-McDonald Lexicon: `data/LM_Word_List.csv`

In [52]:
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
import nltk
nltk.download('omw-1.4')
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from itertools import product

data_filepath ='results/FinancialPhraseBank_DSAll_k-folds/predictions_senti-dd.csv'
senti_dd_lexicon_filepath = 'data/DS50_Entire/Senti-DD.csv'
lm_filepath = 'data/LM_Word_List.csv'

save_filepath = 'results/senti_dd_interpretability_evidence_for_appendix.csv'

[nltk_data] Downloading package omw-1.4 to C:\Users\Jihye
[nltk_data]     Park\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [10]:
df = pd.read_csv(data_filepath)[['headline', 'label']]
lm_df = pd.read_csv(lm_filepath)
senti_dd_lexicon_df = pd.read_csv(senti_dd_lexicon_filepath)

print('sentences\n', df.head(), '\n')
print('Loughran-McDonald\n', lm_df.head(), '\n')
print('Senti-DD\n', senti_dd_lexicon_df.head(), '\n')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Jihye Park\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Jihye Park\AppData\Local\Temp\ipykernel_10916\493411589.py", line 3, in <module>
    senti_dd_lexicon_df = pd.read_csv(senti_dd_lexicon_filepath)
NameError: name 'senti_dd_lexicon_filepath' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Jihye Park\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 1997, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\Jihye Park\anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1112, in structured_traceback
    return FormattedTB.structured_traceback(
  File "C:\Users\Jihye Park\anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1006, in structured_traceback
    return VerboseTB.

In [58]:
def get_intersection(list_one, list_two):
    return list(set(list_one) & set(list_two))

lm_positive_words = lm_df[lm_df['label']=='positive']['word'].values
def detect_lm_positive_words(text):
    tokens = word_tokenize(text)
    return get_intersection(tokens, lm_positive_words)

lm_negative_words = lm_df[lm_df['label']=='negative']['word'].values
def detect_lm_negative_words(text):
    tokens = word_tokenize(text)
    return get_intersection(tokens, lm_negative_words)

lemmatizer=WordNetLemmatizer()
entity_list = list(senti_dd_lexicon_df['entity'].unique())
def detect_entities(text):
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in word_tokenize(text)]
    return get_intersection(lemmatized_tokens, entity_list)

stemmer = PorterStemmer()
directional_word_list = list(senti_dd_lexicon_df['directional_word'].unique())
def detect_directional_words(text):
    stemmed_tokens = [stemmer.stem(token) for token in word_tokenize(text)]
    return get_intersection(stemmed_tokens, directional_word_list)

senti_dd_positive_pair_list = ['_'.join((item1, item2)) for item1, item2 in list(senti_dd_lexicon_df[(senti_dd_lexicon_df['sentiment']=='positive')][['entity', 'directional_word']].values)]
def get_positive_pairs(entities, directional_words):
    this_pair_list = ['_'.join((item1, item2)) for item1, item2 in list(product(entities, directional_words))]
    return get_intersection(this_pair_list, senti_dd_positive_pair_list)

senti_dd_negative_pair_list = ['_'.join((item1, item2)) for item1, item2 in list(senti_dd_lexicon_df[(senti_dd_lexicon_df['sentiment']=='negative')][['entity', 'directional_word']].values)]
def get_negative_pairs(entities, directional_words):
    this_pair_list = ['_'.join((item1, item2)) for item1, item2 in list(product(entities, directional_words))]
    return get_intersection(this_pair_list, senti_dd_negative_pair_list)

def score_to_label(score):
    if score > 0: return 'positive'
    elif score < 0: return 'negative'
    else: return 'neutral'

df['lm_positive_words'] = df['headline'].progress_apply(lambda x: detect_lm_positive_words(x))
df['lm_negative_words'] = df['headline'].progress_apply(lambda x: detect_lm_negative_words(x))
df['detected_entities'] = df['headline'].progress_apply(lambda x: detect_entities(x))
df['detected_directional_words'] = df['headline'].progress_apply(lambda x: detect_directional_words(x))
df['senti_dd_pos_pairs'] = df.progress_apply(lambda x: get_positive_pairs(x['detected_entities'], x['detected_directional_words']), axis=1)
df['senti_dd_neg_pairs'] = df.progress_apply(lambda x: get_negative_pairs(x['detected_entities'], x['detected_directional_words']), axis=1)

df['senti_dd_score'] = df.progress_apply(lambda x: len(x['senti_dd_pos_pairs']) - len(x['senti_dd_neg_pairs']), axis=1)
df['senti_dd_prediction'] = df['senti_dd_score'].progress_apply(lambda x: score_to_label(x))
df['correct'] = df.apply(lambda x: x['label']==x['senti_dd_prediction'], axis=1)
df.head()

100%|██████████████████████████████████████████████████████████████████████████| 2259/2259 [00:00<00:00, 217484.57it/s]


,headline,label,lm_positive_words,lm_negative_words,detected_entities,detected_directional_words,senti_dd_pos_pairs,senti_dd_neg_pairs,senti_dd_score,senti_dd_prediction,correct
0,finnish developer and manufacturer of mobile p...,negative,[],[],"[manufacturer, sale, year, phone, net]",[],[],[],0,neutral,False
1,the transaction is expected to be finalized by...,neutral,[],[],[],[],[],[],0,neutral,True
2,okmetic board of directors has also decided on...,neutral,[],[],"[management, company, share]",[],[],[],0,neutral,True
3,indigo and somoncom serve 377 000 subscribers ...,neutral,[],[],[share],[],[],[],0,neutral,True
4,to see a slide show of all the newest product ...,neutral,[],[],[product],[],[],[],0,neutral,True


Save

In [59]:
df.to_csv(save_filepath, index=False)
print('Created {}'.format(save_filepath))

Created results/senti_dd_interpretability_evidence_for_appendix.csv


In [64]:
appendix_df = df[(df['detected_entities'].apply(lambda x: len(x)!=0)) & \
                 (df['detected_directional_words'].apply(lambda x: len(x)!=0)) & \
                 (df['lm_positive_words'].apply(lambda x: len(x)==0))& \
                 (df['lm_negative_words'].apply(lambda x: len(x)==0))]

result = pd.concat([appendix_df[appendix_df['correct']==True].sample(10), \
                   appendix_df[appendix_df['correct']==False].sample(10)])[['headline','label', 'senti_dd_prediction', 'senti_dd_pos_pairs', 'senti_dd_neg_pairs']]

result

,headline,label,senti_dd_prediction,senti_dd_pos_pairs,senti_dd_neg_pairs
1921,in july september 2008 yit s net sales increas...,positive,positive,"[period_increas, eur_increas, september_increa...",[]
2074,adp news feb 12 2009 finnish construction comp...,negative,negative,"[company_decreas, construction_decreas]","[profit_decreas, net_decreas, news_decreas, eu..."
576,revenue was slightly down at x20ac 495 million...,negative,negative,[],"[year_down, revenue_down]"
1126,operating profit totaled eur 9 4 mn down from ...,negative,negative,[],"[profit_down, eur_down]"
60,finnish metal products company componenta oyj ...,positive,positive,"[product_rose, period_rose, mln_rose, oyj_rose...",[company_rose]
441,tietoenator was down 1 13 pct to 18 38 extendi...,negative,negative,[],"[quarter_down, report_down]"
438,pre tax profit totaled eur 397 4 mn up from eu...,positive,positive,"[eur_up, profit_up]",[]
520,operating profit was eur 0 1 mn down from eur ...,negative,negative,[],"[profit_down, eur_down]"
2064,lithuanian beer makers sold 256 88 million lit...,positive,positive,"[maker_rise, beer_rise, year_rise, cent_rise, ...",[]
1497,operating profit excluding non recurring items...,negative,negative,[],"[item_down, period_down, profit_down, eur_down]"


In [65]:
save_result_filepath = save_filepath.replace('.csv', '_{}.csv'.format(len(result)))
result.to_csv(save_result_filepath, index=False)
print('Created {}'.format(save_result_filepath))

Created results/senti_dd_interpretability_evidence_for_appendix_20.csv
